<h1 style='font-family: "Courier"; font-size: 40px;'> Converting Video to image </h1>
<p style='font-size: 20px;'> Data is given in two seperate folders (Fight and No fight) videos. However, I need to split each videos into their respected folders containing all of the frames for each video </p>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

nonviolet_path = r'VIRAT_footage\Real Life Violence Dataset\NonViolence'
violet_path = r'VIRAT_footage\Real Life Violence Dataset\Violence'


def video_to_image(nonviolet_path, violet_path, path):
    frameWidth, frameHeight = 640, 480
    realLife_directoy = r'VIRAT_footage\Real Life Violence Dataset'
    all_violent_videos = os.listdir(violet_path)
    all_nonviolent_videos = os.listdir(nonviolet_path)
    
    index = 0
    # Store each video into a VideoCapture object
    # Read Each Image and then convert into grayscale
    # Run Threshold and convert into binary image
    # Define a min and max contour area so that not all objects of the video get detected
    # Draw a rectangle around wanted objects
    # Open directory and store image inside
    for video_name in path[:100]:
        cap = cv2.VideoCapture(violet_path + f'\{video_name}')

        while True:
            success, img = cap.read()
            if not img is None:
                img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (frameWidth, frameHeight))
                img_gray = cv2.resize(img_gray, (frameWidth, frameHeight))
                
                # Apply contours --> Maybe threshholding?
                thresh = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 3)
                
                
                cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                cnts = cnts[0] if len(cnts) == 2 else cnts[1]
                
                min_area = 1000
                max_area = 2000
                for c in cnts:
                    area = cv2.contourArea(c)

                    if area < max_area and area > min_area:
                        x, y, w, h = cv2.boundingRect(c)

                        cv2.rectangle(img_gray, (x, y), (x+w, y+h), (255, 255, 255), 2)
                    
                        cv2.imwrite(os.path.join(realLife_directoy, f'{index}.png'), img)
                        cv2.imshow('Result', img) 
                        index += 1
            else:
                
                cap.release()
                cv2.destroyAllWindows()
                break
            
            if cv2.waitKey(1) and 0xFF == ord('q'):
                break

video_to_image(nonviolet_path, violet_path)

## Importing Modules and Defining File Paths

In [ ]:
from copy import deepcopy

noFight_path = r'VIRAT_footage\fight-detection-surv-dataset-master\noFight'
fight_path = r'VIRAT_footage\fight-detection-surv-dataset-master\fight'

frameWidth, frameHeight = 640, 480
realLife_directoy = r'VIRAT_footage\fight-detection-surv-dataset-master'
fight_videos = os.listdir(fight_path)
noFight_videos = os.listdir(noFight_path)
frames = r'VIRAT_footage\fight-detection-surv-dataset-master\frames'

temp_fight = deepcopy(fight_videos)
temp_nofight = deepcopy(noFight_videos)

# Converting Each Frame into 640, 480 Frames and Labeling

In [ ]:
def get_index(path):
    return len(os.listdir(path))


def write_frame(path, type_lst, type):
    #fight_frames = r'C:\Users\Memo\Desktop\For Fun Programs\OpenCv Tutorial\VIRAT_footage\fight-detection-surv-dataset-master\frames\fight_frames'
    noFight_frames = r'C:\Users\Memo\Desktop\For Fun Programs\OpenCv Tutorial\VIRAT_footage\fight-detection-surv-dataset-master\frames\noFight_frames'


    index = get_index(frames)
    
    for video_type in type_lst[:100]:
        cap = cv2.VideoCapture(path + f'\{video_type}')
        fight_videos.pop(0)
        
        while True:
            _, img = cap.read()
            if not img is None:
                img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                img_gray = cv2.resize(img_gray, (frameWidth, frameHeight))
                

                #cv2.imwrite(os.path.join((fight_frames if type.lower() == 'fight' else noFight_frames), f'{type}{index}.png'), img_gray)
                cv2.imwrite(os.path.join(noFight_frames, f'{type}{index}.png'), img_gray)
                cv2.imshow('Frame', img_gray)
                index += 1
            else:
                cap.release()
                cv2.destroyAllWindows()
                break
            if cv2.waitKey(1) and 0xFF == ord('q'):
                break


response = input('Write FIGHT or NOFIGHT videos:')           
declare_path = (fight_path if response.lower() == 'fight' else noFight_path)
declare_videos = (fight_videos if response.lower() == 'fight' else noFight_videos)

write_frame(declare_path, declare_videos, response)

# Split data into train-test-val to prepare for training

In [ ]:
import splitfolders

root = r'VIRAT_footage\fight-detection-surv-dataset-master\frames'

# Splitfolders takes a root directory and then outputs a train-test-val folder into each directory inside the root
splitfolders.ratio(root, output="train-test-val", seed=1337, ratio=(0.70, 0.15, 0.15))

# Create an Image dataset

In [ ]:
import random
import cv2
import numpy as np

data_dir = r'VIRAT_footage\fight-detection-surv-dataset-master\frames'
categories = ['fight_frames', 'noFight_frames']

frameWidth = 640
frameHeight = 480


for category in categories:
    path = os.path.join(data_dir, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_UNCHANGED)

training_data = []

# Takes each image inside the directory and converts to a numpy array
# Then takes the 2D Numpy array (or image) and stores them into a X variable
# X is 3D numpy array made up of 2D np arrays
def create_training_data():
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_UNCHANGED)
                new_array = cv2.resize(img_array, (frameWidth, frameHeight)) 
                training_data.append([new_array, class_num])
            except Exception as e:
                pass


# Given x (which should store float numpy arrays), normalize each array.

create_training_data()

random.shuffle(training_data)

# Create features and labels
X = []
y = []

for features, labels in training_data:
    X.append(features)
    y.append(labels)
    
x = np.array(X).reshape((-1, frameHeight, frameWidth, 1))
y = np.asarray(y).astype('float32')

# Splitting and Normalizing Data to remove noise

In [ ]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(x, y, random_state=42, test_size=0.33)

def normalize(trainX, testX):
    trainX = trainX.astype('float32')
    testX = testX.astype('float32')
    trainX = trainX / 255.0
    testX = testX / 255.0
    
    return trainX, testX

trainX, testX = normalize(trainX, testX)

# Plotting an Example

In [ ]:
import matplotlib.pyplot as plt

for i in range(1):
    plt.figure(figsize=(40,40))
    plt.subplot(3, 3, i+1), plt.imshow(trainX[i], cmap='gray')
    plt.title(('fight' if trainY[i] == 0 else 'no fight'), color="WHITE"), plt.xticks([]), plt.yticks([])
    
plt.tight_layout()
plt.show()

# Building and Training the Model

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten 

def create_model():
    model = Sequential()
    
    model.add(Conv2D(16, (3,3), activation='relu'))
    model.add(Conv2D(16, (3,3), activation='relu'))
    model.add(MaxPooling2D((3,3)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D((3,3)))
    model.add(Dropout(0.25))
    
    
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((3,3)))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    
    opt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    history = model.fit(trainX, trainY, epochs=4, batch_size=16, validation_data=(testX, testY), verbose=1)
    
    save_model(model, r'VIRAT_footage\fight-detection-surv-dataset-master\cctv_v2.h5')
    return history
        
history = create_model()

# Visualizing Train Loss vs val_loss and Val_acc vs Acc

In [ ]:
import tensorflow
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

#model = load_model(r'C:\Users\Memo\Desktop\For Fun Programs\OpenCv Tutorial\VIRAT_footage\fight-detection-surv-dataset-master\cctv.h5')
#model.summary()

def training_vs_validation_loss():
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1,5)
    plt.plot(epochs, loss, 'g', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and Validation Loss'), plt.xlabel('Epochs'), plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
def training_vs_validation_accuracy():
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs= range(1,5)
    plt.plot(epochs, acc, 'g', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and Validation Accuracy'), plt.xlabel('epochs'), plt.ylabel('Accuracy')
    plt.legend()
    plt.show()
    
training_vs_validation_loss()
training_vs_validation_accuracy()

# Converting a Video given to numpy array to pass into trained model

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array


def load_image(filepath):
    frameWidth, frameHeight = 640, 480
    img = load_img(filepath, color_mode='grayscale', target_size=(frameWidth, frameHeight))
    
    # Convert to array
    img = img_to_array(img)
    img = img.reshape(-1, frameHeight, frameWidth, 1)
    
    # Prepare Pixel Data
    img = img.astype('float32')
    img = img / 255.0
    

    return img


def video_to_file(video_path):
    i = 0
    cap = cv2.VideoCapture(video_path)
    while True:
        _, img = cap.read()
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (640, 480)) 
            cv2.imwrite(os.path.join(test_model_dir, f'{i}.png'), img)
            i += 1
        else:
            break

video_to_file(fight_0001) 

# CCTV Live Footage Detection

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model
import time

fight_0001 = r'VIRAT_footage\fight-detection-surv-dataset-master\fight_0001.mpeg'

def load_image(img):
    frameWidth, frameHeight = 640, 480
    #img = load_img(filepath, color_mode='grayscale', target_size=(frameWidth, frameHeight))
    
    # Convert to array
    img = img_to_array(img)
    img = img.reshape(-1, frameHeight, frameWidth, 1)
    
    # Prepare Pixel Data
    img = img.astype('float32')
    img = img / 255.0
    

    return img


def fight_or_nofight(img):
    value = load_image(img)
    big_val = np.argmax(model.predict(value))
    value = np.exp(model.predict(value))[0]
    diff = abs(value[0] - value[1])
    #altercation_text = ('fight' if big_val==1 else 'no fight' if big_val==0 else 'Clear')
    altercation_text = ('fight' if diff<=1.5 else 'no fight' if diff>1.5 else 'Clear')
    print(f'Value: {diff}, Text: {altercation_text}')

    return altercation_text
    

def display(file):
    frameWidth, frameHeight = 640, 480
    
    cam = cv2.VideoCapture(file)
    fourcc = cv2.VideoWriter_fourcc(*'mp4c')
    result = cv2.VideoWriter('output3.mp4', fourcc, 20, (frameWidth, frameHeight))
    
    while True:
        _, img = cam.read()

        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (frameWidth, frameHeight))
            blur = cv2.GaussianBlur(img, (5,5), 0)
            mask = np.zeros(img.shape, np.uint8)
            
            
            #thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 7)
            _, thresh=  cv2.threshold(blur, 80, 255, cv2.THRESH_BINARY_INV)
            dilated = cv2.dilate(thresh, None, iterations=3)
            
            cnts = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            
            
            min_area = 10000
            max_area = 150000
            for c in cnts:
                area = cv2.contourArea(c)
                if area > min_area and area < max_area:                    
                    x, y, w, h = cv2.boundingRect(c)
                    
                    
                    # Get Text for Contour
                    response = fight_or_nofight(img)
                    color = ((0,0,0) if response.lower()=='fight' else (255, 255,255) if response.lower()=='no fight' else (0, 255, 0))
            
                    
                    
                    #img = img[x:x+w, y:y+h]
                    text_image = cv2.rectangle(img, (x,y), (x+w, y+h), (255, 255, 255), 1)
                    cv2.putText(text_image, response, (x,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                    
                   # cv2.imshow('image', img)
                    #result.write(img)
                    cv2.imshow('Thresh', img)


        else:
            cam.release()
            cv2.destroyAllWindows()
            break    
        if cv2.waitKey(1) and 0xFF == ord('q'):
            break

        
        
model_path = r'VIRAT_footage\fight-detection-surv-dataset-master\cctv.h5'
model = load_model(model_path)
display(fight_0001)        

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Value: 1.718275547027588, Text: no fight
Value: 1.7182725667953491, Text: no fight
Value: 1.718275547027588, Text: no fight
Value: 1.7182729244232178, Text: no fight
Value: 1.718275547027588, Text: no fight
Value: 1.7182729244232178, Text: no fight
Value: 1.718275547027588, Text: no fight
Value: 1.7182729244232178, Text: no fight
Value: 1.718275547027588, Text: no fight
Value: 1.7182729244232178, Text: no fight
Value: 1.718275547027588, Text: no fight
Value: 1.7182729244232178, Text: no fight
Value: 1.718275547027588, Text: no fight
Value: 1.7182729244232178, Text: no fight
Value: 1.718275547027588, Text: no fight
Value: 1.7182729244232178, Text: no fight
Value: 1.7182755470

Value: 1.7062500715255737, Text: no fight
Value: 1.7001211643218994, Text: no fight
Value: 1.71040678024292, Text: no fight
Value: 1.705333948135376, Text: no fight
Value: 0.7847898006439209, Text: fight
Value: 0.9401513338088989, Text: fight
Value: 0.7876402139663696, Text: fight
Value: 0.935906171798706, Text: fight
Value: 0.7752536535263062, Text: fight
Value: 0.918825626373291, Text: fight
Value: 0.7754638195037842, Text: fight
Value: 0.9190148115158081, Text: fight
Value: 1.3062132596969604, Text: fight
Value: 1.1940319538116455, Text: fight
Value: 1.3072577714920044, Text: fight
Value: 1.1955375671386719, Text: fight
Value: 1.3796206712722778, Text: fight
Value: 1.3266404867172241, Text: fight
Value: 1.3950365781784058, Text: fight
Value: 1.3457250595092773, Text: fight
Value: 1.2187501192092896, Text: fight
Value: 1.2613469362258911, Text: fight
Value: 1.2076897621154785, Text: fight
Value: 1.251833438873291, Text: fight
Value: 1.2274863719940186, Text: fight
Value: 1.2718359231

Value: 0.36213207244873047, Text: fight
Value: 0.31101059913635254, Text: fight
Value: 1.595321536064148, Text: no fight
Value: 1.5961964130401611, Text: no fight
Value: 1.6175464391708374, Text: no fight
Value: 1.6160727739334106, Text: no fight
Value: 1.6120107173919678, Text: no fight
Value: 1.6119719743728638, Text: no fight
Value: 0.28317511081695557, Text: fight
Value: 0.3198070526123047, Text: fight
Value: 0.3796267509460449, Text: fight
Value: 0.3798699378967285, Text: fight
Value: 0.8492882251739502, Text: fight
Value: 0.8676989078521729, Text: fight
Value: 0.8699444532394409, Text: fight
Value: 0.8691728115081787, Text: fight
Value: 0.7683361768722534, Text: fight
Value: 0.7381627559661865, Text: fight
Value: 1.7151354551315308, Text: no fight
Value: 1.7151579856872559, Text: no fight
Value: 1.7179275751113892, Text: no fight
Value: 1.7179676294326782, Text: no fight
Value: 1.717995047569275, Text: no fight
Value: 1.7180025577545166, Text: no fight
Value: 1.7180886268615723, 

Value: 1.0112249851226807, Text: fight
Value: 1.105127215385437, Text: fight
Value: 0.9989330768585205, Text: fight
Value: 1.0996540784835815, Text: fight
Value: 0.9943422079086304, Text: fight
Value: 1.278711199760437, Text: fight
Value: 0.9516770839691162, Text: fight
Value: 1.2767086029052734, Text: fight
Value: 0.9564144611358643, Text: fight
Value: 1.254691243171692, Text: fight
Value: 0.9032083749771118, Text: fight
Value: 1.2494364976882935, Text: fight
Value: 0.8979223966598511, Text: fight
Value: 1.6953418254852295, Text: no fight
Value: 1.6939414739608765, Text: no fight
Value: 1.6955113410949707, Text: no fight
Value: 1.6936184167861938, Text: no fight
Value: 1.697171688079834, Text: no fight
Value: 1.7153706550598145, Text: no fight
Value: 1.7028807401657104, Text: no fight
Value: 1.7161539793014526, Text: no fight
Value: 1.7019314765930176, Text: no fight
Value: 1.716049313545227, Text: no fight
Value: 1.7013874053955078, Text: no fight
Value: 1.7159628868103027, Text: no 

Value: 0.8761717081069946, Text: fight
Value: 0.9784401655197144, Text: fight
Value: 0.11458766460418701, Text: fight
Value: 1.6893484592437744, Text: no fight
Value: 0.07379293441772461, Text: fight
Value: 1.689537525177002, Text: no fight
Value: 0.00910806655883789, Text: fight
Value: 1.6876661777496338, Text: no fight
Value: 0.030202150344848633, Text: fight
Value: 1.687306523323059, Text: no fight
Value: 0.46014904975891113, Text: fight
Value: 1.5754234790802002, Text: no fight
Value: 0.5876189470291138, Text: fight
Value: 1.5998175144195557, Text: no fight
Value: 0.6275616884231567, Text: fight
Value: 1.59093177318573, Text: no fight
Value: 0.6305105686187744, Text: fight
Value: 1.5932272672653198, Text: no fight
Value: 1.5735043287277222, Text: no fight
Value: 1.7086519002914429, Text: no fight
Value: 1.5456985235214233, Text: no fight
Value: 1.7076252698898315, Text: no fight
Value: 1.547634482383728, Text: no fight
Value: 1.7072224617004395, Text: no fight
Value: 1.550264954566

...

Value: 1.711942434310913, Text: no fight
Value: 1.7030779123306274, Text: no fight
Value: 1.7166420221328735, Text: no fight
Value: 1.7151883840560913, Text: no fight
Value: 1.7166447639465332, Text: no fight
Value: 1.7151387929916382, Text: no fight
Value: 1.7166355848312378, Text: no fight
Value: 1.7152197360992432, Text: no fight
Value: 1.7166433334350586, Text: no fight
Value: 1.7152267694473267, Text: no fight
Value: 1.716915488243103, Text: no fight
Value: 1.7160205841064453, Text: no fight
Value: 1.7167370319366455, Text: no fight
Value: 1.7158076763153076, Text: no fight
Value: 1.7166156768798828, Text: no fight
Value: 1.7157537937164307, Text: no fight
Value: 1.7165892124176025, Text: no fight
Value: 1.71571946144104, Text: no fight
Value: 1.7166038751602173, Text: no fight
Value: 1.7157427072525024, Text: no fight
Value: 1.716591477394104, Text: no fight
Value: 1.7157237529754639, Text: no fight
Value: 1.7165803909301758, Text: no fight
Value: 1.7166979312896729, Text: no fig

Value: 1.7182804346084595, Text: no fight
Value: 1.7182787656784058, Text: no fight
Value: 1.7182787656784058, Text: no fight
Value: 1.7182787656784058, Text: no fight
Value: 1.7182804346084595, Text: no fight
Value: 1.7182804346084595, Text: no fight
Value: 1.7182804346084595, Text: no fight
Value: 1.7182804346084595, Text: no fight
Value: 1.718271255493164, Text: no fight
Value: 1.7182716131210327, Text: no fight
Value: 1.718271255493164, Text: no fight
Value: 1.717816948890686, Text: no fight
Value: 1.7182703018188477, Text: no fight
Value: 1.7177680730819702, Text: no fight
Value: 1.7182719707489014, Text: no fight
Value: 1.717848300933838, Text: no fight
Value: 1.7182670831680298, Text: no fight
Value: 1.717848300933838, Text: no fight
Value: 1.717015266418457, Text: no fight
Value: 1.7122228145599365, Text: no fight
Value: 1.7170426845550537, Text: no fight
Value: 1.7123397588729858, Text: no fight
Value: 1.71462881565094, Text: no fight
Value: 1.7144588232040405, Text: no fight


Value: 1.1796971559524536, Text: fight
Value: 0.9731861352920532, Text: fight
Value: 1.1544228792190552, Text: fight
Value: 0.9715086221694946, Text: fight
Value: 1.1530603170394897, Text: fight
Value: 0.9716391563415527, Text: fight
Value: 1.1530134677886963, Text: fight
Value: 0.9716391563415527, Text: fight
Value: 1.1530134677886963, Text: fight
Value: 1.0373841524124146, Text: fight
Value: 1.2048406600952148, Text: fight
Value: 1.0119823217391968, Text: fight
Value: 1.1887145042419434, Text: fight
Value: 0.999049186706543, Text: fight
Value: 1.2187519073486328, Text: fight
Value: 1.0025659799575806, Text: fight
Value: 1.221917748451233, Text: fight
Value: 1.0347601175308228, Text: fight
Value: 1.2468699216842651, Text: fight
Value: 1.0307964086532593, Text: fight
Value: 1.243862271308899, Text: fight
Value: 1.0761287212371826, Text: fight
Value: 1.3022454977035522, Text: fight
Value: 1.072202205657959, Text: fight
Value: 1.2995121479034424, Text: fight
Value: 1.0373713970184326, Te

Value: 1.5609747171401978, Text: no fight
Value: 1.53037428855896, Text: no fight
Value: 1.5575331449508667, Text: no fight
Value: 1.5626662969589233, Text: no fight
Value: 1.6011070013046265, Text: no fight
Value: 1.575735330581665, Text: no fight
Value: 1.5979444980621338, Text: no fight
Value: 1.5791188478469849, Text: no fight
Value: 1.600584626197815, Text: no fight
Value: 1.587038278579712, Text: no fight
Value: 1.6072453260421753, Text: no fight
Value: 1.5869736671447754, Text: no fight
Value: 1.6071902513504028, Text: no fight
Value: 1.5835742950439453, Text: no fight
Value: 1.604293942451477, Text: no fight
Value: 1.5835742950439453, Text: no fight
Value: 1.604293942451477, Text: no fight
Value: 1.5012058019638062, Text: no fight
Value: 1.5199334621429443, Text: no fight
Value: 1.5024470090866089, Text: no fight
Value: 1.5215305089950562, Text: no fight
Value: 1.6948784589767456, Text: no fight
Value: 1.6719014644622803, Text: no fight
Value: 1.6946516036987305, Text: no fight

Value: 1.7180730104446411, Text: no fight
Value: 1.7180496454238892, Text: no fight
Value: 1.718070387840271, Text: no fight
Value: 1.7180557250976562, Text: no fight
Value: 1.7180756330490112, Text: no fight
Value: 1.7180557250976562, Text: no fight
Value: 1.7180756330490112, Text: no fight
Value: 1.7182139158248901, Text: no fight
Value: 1.7182539701461792, Text: no fight
Value: 1.7182624340057373, Text: no fight
Value: 1.7181822061538696, Text: no fight
Value: 1.7182250022888184, Text: no fight
Value: 1.7182703018188477, Text: no fight
Value: 1.718184232711792, Text: no fight
Value: 1.7182285785675049, Text: no fight
Value: 1.718271255493164, Text: no fight
Value: 1.7181851863861084, Text: no fight
Value: 1.7182295322418213, Text: no fight
Value: 1.718271255493164, Text: no fight
Value: 1.7182751893997192, Text: no fight
Value: 1.7182787656784058, Text: no fight
Value: 1.718275547027588, Text: no fight
Value: 1.7182791233062744, Text: no fight
Value: 1.7160913944244385, Text: no fig